In [4]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
from scipy.sparse import rand as sprand
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.utils import data
from torch.utils.data.dataset import random_split


In [5]:
# The grand dataset
ratings_dataset = pd.read_csv('ratings_small.csv', usecols = ['userId','movieId','rating'])
#ratings
#ratings.movieId
ratings_dataset.rename(columns={'userId':'users',
                          'movieId':'movies',
                          'rating':'ratings'}, 
                 inplace=True)

In [6]:
import torch
from torch.utils import data

# Formatting the data 

class MovieLens(Dataset):
  #'Characterizes a dataset for PyTorch'
    def __init__(self,movies,users,ratings):
        'Initialization'
        self.movies=movies
        self.users=users
        self.ratings=ratings
        
    def __len__(self):
     #   'Denotes the total number of samples'
        return len(self.ratings)

    def __getitem__(self, index):
     #   'Generates one sample of data'
        # Select sample
        
        U = self.users[index]
        V =self.movies[index]
        y = self.ratings[index]
        #obs = {'movieId':movieId,'userId':userId,'rating':rating}
        #obs = self.movieLens.drop('timestamp',axis=1)
        #obs = obs.iloc[index,:].as_matrix()

        return [movies,users,ratings]
    

In [7]:

users = torch.LongTensor(ratings_dataset.users.values)
movies = torch.LongTensor(ratings_dataset.movies.values)
ratings = torch.FloatTensor(ratings_dataset.ratings.values)

Movielens_dataset = MovieLens(users, movies, ratings)

#x = MovieLens(csv_file='ratings_small.csv',root_dir=None)
#print(x[1])
#print(len(x))

In [38]:
shuffle_dataset = True
train_split=0.7
validation_split=0.2
test_split=0.1
random_seed= 42
dataset_size = len(Movielens_dataset)
indices = list(range(dataset_size))

               
split_1 = int(np.floor(train_split * dataset_size))
split_2 = split_1+int(np.floor(validation_split * dataset_size))
split_3 = split_2+int(np.floor(test_split * dataset_size))
split_2end=split_3-split_2

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices, test_indices = indices[:split_1], indices[split_1+1:-split_2end], indices[split_2+1:]

#train_dataset, test_dataset, val_dataset = torch.utils.data.random_split(x, [train_size, test_size,val_size])
#train_dataset,val_dataset, test_dataset = random_split(Movielens_dataset, [train_indices, val_indices,test_indices])

train_dataset,  val_dataset, test_dataset= Movielens_dataset[train_indices], Movielens_dataset[val_indices], Movielens_dataset[test_indices]

print(len(train_indices) + len(val_indices)+len(test_indices))
print(len(Movielens_dataset))

TypeError: slice indices must be integers or None or have an __index__ method

In [35]:
len(train_dataset),len(val_dataset)

(3, 3)

In [36]:
print(train_dataset)

[tensor([ 3.1000e+01,  1.0290e+03,  1.0610e+03,  ...,  6.3650e+03,
         6.3850e+03,  6.5650e+03]), tensor([   1,    1,    1,  ...,  671,  671,  671]), tensor([ 2.5000,  3.0000,  3.0000,  ...,  4.0000,  2.5000,  3.5000])]


In [8]:
train_dataloader = DataLoader(train_dataset,batch_size = 4,shuffle=True)

print(train_dataloader)

#for i_batch, sample_batched in enumerate(train_dataloader):
#    print(i_batch)
#    print(sample_batched)


In [7]:
test_dataloader = DataLoader(test_dataset,batch_size = 4,shuffle=True)


In [8]:
valid_dataloader = DataLoader(val_dataset,batch_size = 4,shuffle=True)


In [9]:
# instantiating model class
num_users = len(ratings_dataset.users[train_indices].unique())
num_items = len(ratings_dataset.movies[train_indices].unique())
print(num_users, num_items)

671 8010


In [10]:
ratings.size

<function Tensor.size>

In [11]:
class MF_bias(nn.Module):
    def __init__(self, num_users, num_items, emb_size=100):
        super(MF_bias, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.user_bias = nn.Embedding(num_users, 1)
        self.item_emb = nn.Embedding(num_items, emb_size)
        self.item_bias = nn.Embedding(num_items, 1)
        # init 
        self.user_emb.weight.data.uniform_(0,0.05)
        self.item_emb.weight.data.uniform_(0,0.05)
        self.user_bias.weight.data.uniform_(-0.01,0.01)
        self.item_bias.weight.data.uniform_(-0.01,0.01)
        
    def forward(self,userId,movieId):
        U = self.user_emb(users)
        V = self.item_emb(movies)
        b_u = self.user_bias(users).squeeze()
        b_v = self.item_bias(movies).squeeze()
        return (U*V).sum(1) +  b_users  + b_movies

In [12]:
model = MF_bias(num_users, num_items, emb_size=100)

In [13]:
def train_epocs(model, epochs=10, lr=0.01, wd=0.0, unsqueeze=False):
    parameters = filter(lambda p: p.requires_grad, model.parameters()) # get all parameters which need grad
    optimizer = torch.optim.Adam(parameters, lr=lr, weight_decay=wd)
    model.train() # into training mode
    for i in range(epochs):
        running_loss = 0.0
        
        for j, data in enumerate(train_dataloader):
            users, items, ratings = data
            users = Variable(users)
            items = Variable(items)
            ratings = Variable(ratings).float()
            
            if unsqueeze:
                ratings = ratings.unsqueeze(1)
            y_hat = model(users, items)
            loss = F.mse_loss(y_hat, ratings)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.data[0]
        print("training loss for epoch ",i+1, ": ", running_loss/j+1) # used to be loss.data[0]
    test_loss(model, unsqueeze)


In [14]:
len(valid_dataloader), len(train_dataloader), len(test_dataloader)

(1, 1, 1)

In [15]:

# defining test loss which has been used in train_epochs

def test_loss(model, unsqueeze=False):
    model.eval() # go to evaluation mode
    
    running_loss = 0.
    for j, data in enumerate(valid_dataloader):
        users, items, ratings = data
        users, items, ratings = Variable(users), Variable(items), Variable(ratings).float()
        
        users = users.gpu() # put on gpu
        items = items.gpu()
        ratings = ratings.gpu()       
        if unsqueeze:
            ratings = ratings.unsqueeze(1)
        y_hat = model(users, items)
        loss_now = F.mse_loss(y_hat, ratings)
        running_loss+= loss_now.data[0]
    print("validation loss", ": ", running_loss/len(valid_dataloader)) # j means this many iterations till end

In [18]:
train_epocs(model, epochs=10, lr=0.01)

RuntimeError: Expected a Tensor of type torch.LongTensor but found a type torch.FloatTensor for sequence element 1  in sequence argument at position #1 'tensors'

In [ ]:
###NEDESTÅENDE ER GAMMEL LORT ### 

In [ ]:

user=ratings.userId
user = user.values

item = ratings.movieId
item = item.values


#ratings = torch.tensor(ratings.values)
#ratings=ratings.toarray()
print(user)
print(item)
n_users = len(user)
n_items= len(item)


ratings = ratings.pivot(index='userId', columns='movieId', values='rating')
mat = ratings.values
mat[np.isnan(mat)] = 0
#mat=mat.astype(int)
print(mat)

In [ ]:
class MatrixFactorization(torch.nn.Module):
    
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, 
                                               n_factors,
                                               sparse=True)
        #print(self.user_factors)
        self.item_factors = torch.nn.Embedding(n_items, 
                                               n_factors,
                                               sparse=True)
        
    def forward(self, user, item):
        return (self.user_factors(user) * self.item_factors(item)).sum(1)

In [ ]:
model = MatrixFactorization(n_users, n_items, n_factors=20)
loss_func = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), 
                            lr=1e-6) # learning rate

In [ ]:
rows, cols = mat.nonzero()
print(rows)
print(cols)
p = np.random.permutation(len(rows))
print(p)
rows, cols = rows[p], cols[p]
print(rows)
print(cols)

for row, col in zip(*(rows, cols)):
    # Turn data into variables
    rating = Variable(torch.FloatTensor([mat[row, col]]))
    row = Variable(torch.LongTensor([np.long(row)]))
    col = Variable(torch.LongTensor([np.long(col)]))
    
    # Predict and calculate loss
    prediction = model(row, col)
    loss = loss_func(prediction, rating)
    
    # Backpropagate
    loss.backward()
    
    # Update the parameters
    optimizer.step()
    
    print(rating)
    print(row)
    print(col)
    print(prediction)
    print(loss)
  